# Project 2 - GeoTweet

@Author Jeffery Brown (daddyjab)<br>
@Date 3/25/19<br>
@File GeoTweet_Queries


In [19]:
# import necessary libraries
import os
from flask import Flask, render_template, jsonify, request, redirect

from pprint import pprint

#################################################
# Flask Setup
#################################################
app = Flask(__name__)

#################################################
# Database Setup
#################################################

from flask_sqlalchemy import SQLAlchemy
from sqlalchemy.sql.expression import func

#Probably don't need these from SQLAlchemy: asc, desc, between, distinct, func, null, nullsfirst, nullslast, or_, and_, not_

db_path_flask_app = "sqlite:///../../python/data/twitter_trends.db"
app.config['SQLALCHEMY_DATABASE_URI'] = os.environ.get('DATABASE_URL', '') or db_path_flask_app
# app.config['SQLALCHEMY_DATABASE_URI'] = db_path_flask_app

In [20]:
# Flask-SQLAlchemy database
db = SQLAlchemy(app)

C:\Users\Jeff\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\flask_sqlalchemy\__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [21]:
# Database schema for Twitter 'locations' table
class Location(db.Model):
    __tablename__ = 'locations'
    
    # Defining the columns for the table 'locations',
    # which will hold all of the locations in the U.S. for which
    # top trends data is available, as well as location specific
    # info like latitude/longitude
    id = db.Column(db.Integer, primary_key=True)
    woeid = db.Column(db.Integer)
    twitter_country = db.Column(db.String(100))
    tritter_country_code = db.Column(db.String(10))
    twitter_name = db.Column(db.String(250))
    twitter_parentid = db.Column(db.Integer)
    twitter_type = db.Column(db.String(50))
    country_name = db.Column(db.String(250))
    country_name_only = db.Column(db.String(250))
    country_woeid = db.Column(db.Integer)
    county_name = db.Column(db.String(250))
    county_name_only = db.Column(db.String(250))
    county_woeid = db.Column(db.Integer)
    latitude = db.Column(db.Float)
    longitude = db.Column(db.Float)
    name_full = db.Column(db.String(250))
    name_only = db.Column(db.String(250))
    name_woe = db.Column(db.String(250))
    place_type = db.Column(db.String(250))
    state_name = db.Column(db.String(250))
    state_name_only = db.Column(db.String(250))
    state_woeid = db.Column(db.Integer)
    timezone = db.Column(db.String(250))
    
    def __repr__(self):
        return '<Location %r>' % (self.name)

# Database schema for Twitter 'trends' table
class Trend(db.Model):
    __tablename__ = 'trends'
    
    # Defining the columns for the table 'trends',
    # which will hold all of the top trends associated with
    # locations in the 'locations' table
    id = db.Column(db.Integer, primary_key=True)
    woeid = db.Column(db.Integer, db.ForeignKey('locations.woeid') )
    twitter_as_of = db.Column(db.String(100))
    twitter_created_at = db.Column(db.String(100))
    twitter_name = db.Column(db.String(250))
    twitter_tweet_name = db.Column(db.String(250))
    twitter_tweet_promoted_content = db.Column(db.String(250))
    twitter_tweet_query = db.Column(db.String(250))
    twitter_tweet_url = db.Column(db.String(250))
    twitter_tweet_volume = db.Column(db.Float)

    locations = db.relationship('Location', backref=db.backref('trends', lazy=True))
     
    def __repr__(self):
        return '<Trend %r>' % (self.name)


In [22]:
# Return a list of all locations with Twitter Top Trend info
results = db.session.query(Location).all()

loc_list = []
for r in results:
    loc_info = {
        'woeid': r.woeid,
        'latitude': r.latitude,
        'longitude': r.longitude,
        'name_full': r.name_full,
        'name_only': r.name_only,
        'name_woe': r.name_woe,
        'county_name': r.county_name,
        'county_name_only': r.county_name_only,
        'county_woeid': r.county_woeid,
        'state_name': r.state_name,
        'state_name_only': r.state_name_only,
        'state_woeid': r.state_woeid,
        'country_name': r.country_name,
        'country_name_only': r.country_name_only,
        'country_woeid': r.country_woeid,
        'place_type': r.place_type,
        'timezone': r.timezone,
        'twitter_type': r.twitter_type,
        'twitter_country': r.twitter_country,
        'tritter_country_code': r.tritter_country_code,
        'twitter_name': r.twitter_name,
        'twitter_parentid': r.twitter_parentid
    }

    # loc_info = {
    #     'woeid': r.Location.woeid,
    #     'latitude': r.Location.latitude,
    #     'longitude': r.Location.longitude,
    #     'name_full': r.Location.name_full,
    #     'name_only': r.Location.name_only,
    #     'name_woe': r.Location.name_woe,
    #     'county_name': r.Location.county_name,
    #     'county_name_only': r.Location.county_name_only,
    #     'county_woeid': r.Location.county_woeid,
    #     'state_name': r.Location.state_name,
    #     'state_name_only': r.Location.state_name_only,
    #     'state_woeid': r.Location.state_woeid,
    #     'country_name': r.Location.country_name,
    #     'country_name_only': r.Location.country_name_only,
    #     'country_woeid': r.Location.country_woeid,
    #     'place_type': r.Location.place_type,
    #     'timezone': r.Location.timezone,
    #     'twitter_type': r.Location.twitter_type,
    #     'twitter_country': r.Location.twitter_country,
    #     'tritter_country_code': r.Location.tritter_country_code,
    #     'twitter_parentid': r.Location.twitter_parentid,

    #     'twitter_as_of': r.Trend.twitter_as_of,
    #     'twitter_created_at': r.Trend.twitter_created_at,
    #     'twitter_name': r.Trend.twitter_name,
    #     'twitter_tweet_name': r.Trend.twitter_tweet_name,
    #     'twitter_tweet_promoted_content': r.Trend.twitter_tweet_promoted_content,
    #     'twitter_tweet_query': r.Trend.twitter_tweet_query,
    #     'twitter_tweet_url': r.Trend.twitter_tweet_url,
    #     'twitter_tweet_volume': r.Trend.twitter_tweet_volume
    # }

    loc_list.append(loc_info)

In [23]:
pprint(len(loc_list))

64


In [18]:
# Return the full list of all trends with Twitter Top Trend info
results = db.session.query(Trend).all()

trend_list = []
for r in results:
    trend_info = {
        'woeid': r.woeid,
        'twitter_as_of': r.twitter_as_of,
        'twitter_created_at': r.twitter_created_at,
        'twitter_name': r.twitter_name,
        'twitter_tweet_name': r.twitter_tweet_name,
        'twitter_tweet_promoted_content': r.twitter_tweet_promoted_content,
        'twitter_tweet_query': r.twitter_tweet_query,
        'twitter_tweet_url': r.twitter_tweet_url,
        'twitter_tweet_volume': r.twitter_tweet_volume
    }

    trend_list.append(trend_info)
    
pprint(trend_list)

[{'twitter_as_of': '2019-04-22T02:41:03Z',
  'twitter_created_at': '2019-04-22T02:32:13Z',
  'twitter_name': 'Albuquerque',
  'twitter_tweet_name': '#GameofThrones',
  'twitter_tweet_promoted_content': None,
  'twitter_tweet_query': '%23GameofThrones',
  'twitter_tweet_url': 'http://twitter.com/search?q=%23GameofThrones',
  'twitter_tweet_volume': 715597.0,
  'woeid': 2352824},
 {'twitter_as_of': '2019-04-22T02:41:03Z',
  'twitter_created_at': '2019-04-22T02:32:13Z',
  'twitter_name': 'Albuquerque',
  'twitter_tweet_name': 'Happy Easter',
  'twitter_tweet_promoted_content': None,
  'twitter_tweet_query': '%22Happy+Easter%22',
  'twitter_tweet_url': 'http://twitter.com/search?q=%22Happy+Easter%22',
  'twitter_tweet_volume': 1846174.0,
  'woeid': 2352824},
 {'twitter_as_of': '2019-04-22T02:41:03Z',
  'twitter_created_at': '2019-04-22T02:32:13Z',
  'twitter_name': 'Albuquerque',
  'twitter_tweet_name': 'Sri Lanka',
  'twitter_tweet_promoted_content': None,
  'twitter_tweet_query': '%22Sri

  'twitter_created_at': '2019-04-22T02:32:13Z',
  'twitter_name': 'Albuquerque',
  'twitter_tweet_name': '#StellarAwards2019',
  'twitter_tweet_promoted_content': None,
  'twitter_tweet_query': '%23StellarAwards2019',
  'twitter_tweet_url': 'http://twitter.com/search?q=%23StellarAwards2019',
  'twitter_tweet_volume': None,
  'woeid': 2352824},
 {'twitter_as_of': '2019-04-22T02:41:03Z',
  'twitter_created_at': '2019-04-22T02:32:13Z',
  'twitter_name': 'Albuquerque',
  'twitter_tweet_name': '#DisneyNight',
  'twitter_tweet_promoted_content': None,
  'twitter_tweet_query': '%23DisneyNight',
  'twitter_tweet_url': 'http://twitter.com/search?q=%23DisneyNight',
  'twitter_tweet_volume': None,
  'woeid': 2352824},
 {'twitter_as_of': '2019-04-22T02:41:03Z',
  'twitter_created_at': '2019-04-22T02:32:13Z',
  'twitter_name': 'Albuquerque',
  'twitter_tweet_name': '#KUWTK',
  'twitter_tweet_promoted_content': None,
  'twitter_tweet_query': '%23KUWTK',
  'twitter_tweet_url': 'http://twitter.com/sea

  'twitter_tweet_volume': None,
  'woeid': 2357024},
 {'twitter_as_of': '2019-04-22T02:41:03Z',
  'twitter_created_at': '2019-04-22T02:32:09Z',
  'twitter_name': 'Atlanta',
  'twitter_tweet_name': '#VeryScaryPeople',
  'twitter_tweet_promoted_content': None,
  'twitter_tweet_query': '%23VeryScaryPeople',
  'twitter_tweet_url': 'http://twitter.com/search?q=%23VeryScaryPeople',
  'twitter_tweet_volume': None,
  'woeid': 2357024},
 {'twitter_as_of': '2019-04-22T02:41:04Z',
  'twitter_created_at': '2019-04-22T02:32:10Z',
  'twitter_name': 'Austin',
  'twitter_tweet_name': 'Arya',
  'twitter_tweet_promoted_content': None,
  'twitter_tweet_query': 'Arya',
  'twitter_tweet_url': 'http://twitter.com/search?q=Arya',
  'twitter_tweet_volume': 283590.0,
  'woeid': 2357536},
 {'twitter_as_of': '2019-04-22T02:41:04Z',
  'twitter_created_at': '2019-04-22T02:32:10Z',
  'twitter_name': 'Austin',
  'twitter_tweet_name': '#GameOfThornes',
  'twitter_tweet_promoted_content': None,
  'twitter_tweet_query'

  'twitter_tweet_name': 'Arya',
  'twitter_tweet_promoted_content': None,
  'twitter_tweet_query': 'Arya',
  'twitter_tweet_url': 'http://twitter.com/search?q=Arya',
  'twitter_tweet_volume': 283590.0,
  'woeid': 2358820},
 {'twitter_as_of': '2019-04-22T02:41:05Z',
  'twitter_created_at': '2019-04-22T02:32:09Z',
  'twitter_name': 'Baltimore',
  'twitter_tweet_name': '#DemThrones',
  'twitter_tweet_promoted_content': None,
  'twitter_tweet_query': '%23DemThrones',
  'twitter_tweet_url': 'http://twitter.com/search?q=%23DemThrones',
  'twitter_tweet_volume': 56705.0,
  'woeid': 2358820},
 {'twitter_as_of': '2019-04-22T02:41:05Z',
  'twitter_created_at': '2019-04-22T02:32:09Z',
  'twitter_name': 'Baltimore',
  'twitter_tweet_name': 'Brienne',
  'twitter_tweet_promoted_content': None,
  'twitter_tweet_query': 'Brienne',
  'twitter_tweet_url': 'http://twitter.com/search?q=Brienne',
  'twitter_tweet_volume': 139079.0,
  'woeid': 2358820},
 {'twitter_as_of': '2019-04-22T02:41:05Z',
  'twitter_

 {'twitter_as_of': '2019-04-22T02:41:06Z',
  'twitter_created_at': '2019-04-22T02:32:10Z',
  'twitter_name': 'Baton Rouge',
  'twitter_tweet_name': '#ForeverLSU',
  'twitter_tweet_promoted_content': None,
  'twitter_tweet_query': '%23ForeverLSU',
  'twitter_tweet_url': 'http://twitter.com/search?q=%23ForeverLSU',
  'twitter_tweet_volume': None,
  'woeid': 2359991},
 {'twitter_as_of': '2019-04-22T02:41:06Z',
  'twitter_created_at': '2019-04-22T02:32:10Z',
  'twitter_name': 'Baton Rouge',
  'twitter_tweet_name': 'Jesus',
  'twitter_tweet_promoted_content': None,
  'twitter_tweet_query': 'Jesus',
  'twitter_tweet_url': 'http://twitter.com/search?q=Jesus',
  'twitter_tweet_volume': 1284481.0,
  'woeid': 2359991},
 {'twitter_as_of': '2019-04-22T02:41:06Z',
  'twitter_created_at': '2019-04-22T02:32:10Z',
  'twitter_name': 'Baton Rouge',
  'twitter_tweet_name': 'the tigers',
  'twitter_tweet_promoted_content': None,
  'twitter_tweet_query': '%22the+tigers%22',
  'twitter_tweet_url': 'http://t

...

  'twitter_tweet_volume': 48326.0,
  'woeid': 2503863},
 {'twitter_as_of': '2019-04-22T02:42:04Z',
  'twitter_created_at': '2019-04-22T02:32:10Z',
  'twitter_name': 'Tampa',
  'twitter_tweet_name': 'Theon',
  'twitter_tweet_promoted_content': None,
  'twitter_tweet_query': 'Theon',
  'twitter_tweet_url': 'http://twitter.com/search?q=Theon',
  'twitter_tweet_volume': 34149.0,
  'woeid': 2503863},
 {'twitter_as_of': '2019-04-22T02:42:04Z',
  'twitter_created_at': '2019-04-22T02:32:10Z',
  'twitter_name': 'Tampa',
  'twitter_tweet_name': 'Podrick',
  'twitter_tweet_promoted_content': None,
  'twitter_tweet_query': 'Podrick',
  'twitter_tweet_url': 'http://twitter.com/search?q=Podrick',
  'twitter_tweet_volume': 15645.0,
  'woeid': 2503863},
 {'twitter_as_of': '2019-04-22T02:42:04Z',
  'twitter_created_at': '2019-04-22T02:32:10Z',
  'twitter_name': 'Tampa',
  'twitter_tweet_name': 'Ghost',
  'twitter_tweet_promoted_content': None,
  'twitter_tweet_query': 'Ghost',
  'twitter_tweet_url': 'h

  'twitter_tweet_name': 'Ghost',
  'twitter_tweet_promoted_content': None,
  'twitter_tweet_query': 'Ghost',
  'twitter_tweet_url': 'http://twitter.com/search?q=Ghost',
  'twitter_tweet_volume': 84041.0,
  'woeid': 2508428},
 {'twitter_as_of': '2019-04-22T02:42:05Z',
  'twitter_created_at': '2019-04-22T02:40:44Z',
  'twitter_name': 'Tucson',
  'twitter_tweet_name': 'Grey Worm',
  'twitter_tweet_promoted_content': None,
  'twitter_tweet_query': '%22Grey+Worm%22',
  'twitter_tweet_url': 'http://twitter.com/search?q=%22Grey+Worm%22',
  'twitter_tweet_volume': None,
  'woeid': 2508428},
 {'twitter_as_of': '2019-04-22T02:42:05Z',
  'twitter_created_at': '2019-04-22T02:40:44Z',
  'twitter_name': 'Tucson',
  'twitter_tweet_name': 'Raymond Felton',
  'twitter_tweet_promoted_content': None,
  'twitter_tweet_query': '%22Raymond+Felton%22',
  'twitter_tweet_url': 'http://twitter.com/search?q=%22Raymond+Felton%22',
  'twitter_tweet_volume': None,
  'woeid': 2508428},
 {'twitter_as_of': '2019-04-22

 {'twitter_as_of': '2019-04-22T02:42:06Z',
  'twitter_created_at': '2019-04-22T02:40:45Z',
  'twitter_name': 'Virginia Beach',
  'twitter_tweet_name': 'Lyanna Mormont',
  'twitter_tweet_promoted_content': None,
  'twitter_tweet_query': '%22Lyanna+Mormont%22',
  'twitter_tweet_url': 'http://twitter.com/search?q=%22Lyanna+Mormont%22',
  'twitter_tweet_volume': None,
  'woeid': 2512636},
 {'twitter_as_of': '2019-04-22T02:42:06Z',
  'twitter_created_at': '2019-04-22T02:40:45Z',
  'twitter_name': 'Virginia Beach',
  'twitter_tweet_name': 'Jorah',
  'twitter_tweet_promoted_content': None,
  'twitter_tweet_query': 'Jorah',
  'twitter_tweet_url': 'http://twitter.com/search?q=Jorah',
  'twitter_tweet_volume': 11491.0,
  'woeid': 2512636},
 {'twitter_as_of': '2019-04-22T02:42:06Z',
  'twitter_created_at': '2019-04-22T02:40:45Z',
  'twitter_name': 'Virginia Beach',
  'twitter_tweet_name': 'Knight',
  'twitter_tweet_promoted_content': None,
  'twitter_tweet_query': 'Knight',
  'twitter_tweet_url':

  'twitter_tweet_query': 'Missandei',
  'twitter_tweet_url': 'http://twitter.com/search?q=Missandei',
  'twitter_tweet_volume': None,
  'woeid': 2514815},
 {'twitter_as_of': '2019-04-22T02:42:07Z',
  'twitter_created_at': '2019-04-22T02:32:09Z',
  'twitter_name': 'Washington',
  'twitter_tweet_name': 'Maisie Williams',
  'twitter_tweet_promoted_content': None,
  'twitter_tweet_query': '%22Maisie+Williams%22',
  'twitter_tweet_url': 'http://twitter.com/search?q=%22Maisie+Williams%22',
  'twitter_tweet_volume': None,
  'woeid': 2514815},
 {'twitter_as_of': '2019-04-22T02:42:07Z',
  'twitter_created_at': '2019-04-22T02:32:09Z',
  'twitter_name': 'Washington',
  'twitter_tweet_name': "Giant's Milk",
  'twitter_tweet_promoted_content': None,
  'twitter_tweet_query': '%22Giant%27s+Milk%22',
  'twitter_tweet_url': 'http://twitter.com/search?q=%22Giant%27s+Milk%22',
  'twitter_tweet_volume': None,
  'woeid': 2514815},
 {'twitter_as_of': '2019-04-22T02:42:07Z',
  'twitter_created_at': '2019-04-

  'twitter_name': 'United States',
  'twitter_tweet_name': 'Tarth',
  'twitter_tweet_promoted_content': None,
  'twitter_tweet_query': 'Tarth',
  'twitter_tweet_url': 'http://twitter.com/search?q=Tarth',
  'twitter_tweet_volume': 35707.0,
  'woeid': 23424977},
 {'twitter_as_of': '2019-04-22T02:42:08Z',
  'twitter_created_at': '2019-04-22T02:32:09Z',
  'twitter_name': 'United States',
  'twitter_tweet_name': 'Giantsbane',
  'twitter_tweet_promoted_content': None,
  'twitter_tweet_query': 'Giantsbane',
  'twitter_tweet_url': 'http://twitter.com/search?q=Giantsbane',
  'twitter_tweet_volume': None,
  'woeid': 23424977},
 {'twitter_as_of': '2019-04-22T02:42:08Z',
  'twitter_created_at': '2019-04-22T02:32:09Z',
  'twitter_name': 'United States',
  'twitter_tweet_name': 'Double OT',
  'twitter_tweet_promoted_content': None,
  'twitter_tweet_query': '%22Double+OT%22',
  'twitter_tweet_url': 'http://twitter.com/search?q=%22Double+OT%22',
  'twitter_tweet_volume': None,
  'woeid': 23424977},
 {

In [39]:
# In the app code for this route, there is no need to do anything special
a_tweet = "#OpeningDay"

In [40]:
# Return a list of all locations that have the specified tweet in its top trends
# and then sort the results by tweet volume in descending order
results = db.session.query(Trend, Location).join(Location) \
                    .filter(Trend.twitter_tweet_name == a_tweet ) \
                    .order_by( Trend.twitter_tweet_volume.desc() ).all()

loc_list = []
for r in results:
    #print(f"Trend Information for {r.Trend.woeid} {r.Location.name_full}: {r.Trend.twitter_tweet_name} {r.Trend.twitter_tweet_volume}")
    loc_info = {
        'woeid': r.Location.woeid,
        'latitude': r.Location.latitude,
        'longitude': r.Location.longitude,
        'name_full': r.Location.name_full,
        'name_only': r.Location.name_only,
        'name_woe': r.Location.name_woe,
        'county_name': r.Location.county_name,
        'county_name_only': r.Location.county_name_only,
        'county_woeid': r.Location.county_woeid,
        'state_name': r.Location.state_name,
        'state_name_only': r.Location.state_name_only,
        'state_woeid': r.Location.state_woeid,
        'country_name': r.Location.country_name,
        'country_name_only': r.Location.country_name_only,
        'country_woeid': r.Location.country_woeid,
        'place_type': r.Location.place_type,
        'timezone': r.Location.timezone,
        'twitter_type': r.Location.twitter_type,
        'twitter_country': r.Location.twitter_country,
        'tritter_country_code': r.Location.tritter_country_code,
        'twitter_parentid': r.Location.twitter_parentid,

        'twitter_as_of': r.Trend.twitter_as_of,
        'twitter_created_at': r.Trend.twitter_created_at,
        'twitter_name': r.Trend.twitter_name,
        'twitter_tweet_name': r.Trend.twitter_tweet_name,
        'twitter_tweet_promoted_content': r.Trend.twitter_tweet_promoted_content,
        'twitter_tweet_query': r.Trend.twitter_tweet_query,
        'twitter_tweet_url': r.Trend.twitter_tweet_url,
        'twitter_tweet_volume': r.Trend.twitter_tweet_volume
    }

    loc_list.append(loc_info)


In [41]:
print(len(loc_list))
pprint(loc_list)

31
[{'country_name': 'United States',
  'country_name_only': 'United States',
  'country_woeid': 23424977,
  'county_name': 'Baltimore City County, Maryland, United States',
  'county_name_only': 'Baltimore City County',
  'county_woeid': 12588679,
  'latitude': 39.3,
  'longitude': -76.61,
  'name_full': 'Baltimore, Maryland, United States',
  'name_only': 'Baltimore',
  'name_woe': 'Baltimore',
  'place_type': 'locality',
  'state_name': 'Maryland, United States',
  'state_name_only': 'Maryland',
  'state_woeid': 2347579,
  'timezone': 'America/New_York',
  'tritter_country_code': 'US',
  'twitter_as_of': '2019-03-30T02:10:09Z',
  'twitter_country': 'United States',
  'twitter_created_at': '2019-03-30T02:07:05Z',
  'twitter_name': 'Baltimore',
  'twitter_parentid': 23424977,
  'twitter_tweet_name': '#OpeningDay',
  'twitter_tweet_promoted_content': None,
  'twitter_tweet_query': '%23OpeningDay',
  'twitter_tweet_url': 'http://twitter.com/search?q=%23OpeningDay',
  'twitter_tweet_volu

  'twitter_as_of': '2019-03-30T02:10:06Z',
  'twitter_country': 'United States',
  'twitter_created_at': '2019-03-30T02:02:12Z',
  'twitter_name': 'Albuquerque',
  'twitter_parentid': 23424977,
  'twitter_tweet_name': '#OpeningDay',
  'twitter_tweet_promoted_content': None,
  'twitter_tweet_query': '%23OpeningDay',
  'twitter_tweet_url': 'http://twitter.com/search?q=%23OpeningDay',
  'twitter_tweet_volume': 52963.0,
  'twitter_type': 'Town',
  'woeid': 2352824},
 {'country_name': 'United States',
  'country_name_only': 'United States',
  'country_woeid': 23424977,
  'county_name': 'Hamilton County, Ohio, United States',
  'county_name_only': 'Hamilton County',
  'county_woeid': 12589557,
  'latitude': 39.141,
  'longitude': -84.505,
  'name_full': 'Cincinnati, Ohio, United States',
  'name_only': 'Cincinnati',
  'name_woe': 'Cincinnati',
  'place_type': 'locality',
  'state_name': 'Ohio, United States',
  'state_name_only': 'Ohio',
  'state_woeid': 2347594,
  'timezone': 'America/New_Y

In [43]:
# URL Library urllib that provides the quote_plus method that can be used
# to parse a string being added to a URL to convert spaces, hashtags, and other
# characters to %xx format
#
# In the function/code that is performing a query to the
# Heroku site to the /locations/tweet/<a_tweet> route
# Use function urllib.parse.quote() to encode the before adding it
# to the URL -- example:
# a_tweet = "#OpeningDay"
# a_tweet_parsed = urllib.parse.quote(a_tweet)
#
# In JavaScript, will use encodeURIComponent('#OpeningDay')
import urllib

# Use the requests module to read JSON from the GeoTweet URL
import requests

In [93]:
geotweet_base_url = "https://geotweetapp.herokuapp.com"
geotweet_loc_tweet_route = "/locations/tweet/"
# geotweet_tweet_name = "#OpeningDay"
# geotweet_tweet_name = "War Eagle"
# geotweet_tweet_name = "#Ultra2019"
geotweet_tweet_name = "#DarknessMakesMeFeel"
geotweet_tweet_name_safe = urllib.parse.quote(geotweet_tweet_name)
print (geotweet_tweet_name_safe)

%23DarknessMakesMeFeel


In [94]:
geotweet_url = geotweet_base_url + geotweet_loc_tweet_route + geotweet_tweet_name_safe

response = requests.get(geotweet_url)
if response.status_code == requests.codes.ok:
    print ("Request was OK")
else:
    print (f"Uh oh, had a problem with the web request: Status Code = '{response.status_code}'")
    
results = response.json()
pprint(len(results))

Request was OK
16


In [96]:
print (geotweet_url)

https://geotweetapp.herokuapp.com/locations/tweet/%23DarknessMakesMeFeel


In [95]:
pprint(results)

# for key, value in results.iteritems():
#     print (key, value)

[{'country_name': 'United States',
  'country_name_only': 'United States',
  'country_woeid': 23424977,
  'county_name': 'Fulton County, Georgia, United States',
  'county_name_only': 'Fulton County',
  'county_woeid': 12587929,
  'latitude': 33.763,
  'longitude': -84.423,
  'name_full': 'Atlanta, Georgia, United States',
  'name_only': 'Atlanta',
  'name_woe': 'Atlanta',
  'place_type': 'locality',
  'state_name': 'Georgia, United States',
  'state_name_only': 'Georgia',
  'state_woeid': 2347569,
  'timezone': 'America/New_York',
  'tritter_country_code': 'US',
  'twitter_as_of': '2019-03-30T02:10:07Z',
  'twitter_country': 'United States',
  'twitter_created_at': '2019-03-30T02:07:05Z',
  'twitter_name': 'Atlanta',
  'twitter_parentid': 23424977,
  'twitter_tweet_name': '#DarknessMakesMeFeel',
  'twitter_tweet_promoted_content': None,
  'twitter_tweet_query': '%23DarknessMakesMeFeel',
  'twitter_tweet_url': 'http://twitter.com/search?q=%23DarknessMakesMeFeel',
  'twitter_tweet_volum